In [1]:
import json
import os
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from main import main

In [2]:
def plot_relationships(entities, relationships):
    G = nx.DiGraph()
    G.add_nodes_from(entities)
    for subj, rel, obj in relationships:
        G.add_edge(subj, obj, label=rel)

    pos = nx.spring_layout(G)
    plt.figure(figsize=(12, 8))
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=2500, font_size=10, edge_color='gray')
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    plt.title("Entity Relationship Graph")
    plt.show()

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import json
from datetime import datetime
import importlib.util

# Widgets
file_picker = widgets.FileUpload(accept=".txt,.png,.jpg,.jpeg,.pdf,.mp3,.mp4", multiple=False)
query_box = widgets.Text(placeholder='Enter your search query here...')
run_button = widgets.Button(description="Run Pipeline")
output = widgets.Output()

def on_run_clicked(b):
    with output:
        clear_output(wait=True)
        output.clear_output()

        if file_picker.value:
            # Handle both tuple and dict edge cases
            uploaded_info = (
                file_picker.value[0]
                if isinstance(file_picker.value, tuple)
                else list(file_picker.value.values())[0]
            )

            fname = uploaded_info['name']
            data = uploaded_info['content']
            path = os.path.join("assets", fname)

            os.makedirs("assets", exist_ok=True)
            with open(path, "wb") as f:
                f.write(data)

            os.system(f'python main.py "{path}" "{query_box.value}"')

            try:
                with open("results/hybrid_results.json") as f:
                    result_data = json.load(f)
            except FileNotFoundError:
                print("Error: hybrid_results.json not found. Check if main.py crashed.")
                return

            query_log = {
                "timestamp": datetime.now().isoformat(),
                "query": query_box.value,
                "source_file": path,
                "entities": result_data.get("entities", []),
                "relationships": result_data.get("relationships", []),
                "vector_results": result_data.get("vector_results", []),
                "graph_results": result_data.get("graph_results", []),
            }

            os.makedirs("results", exist_ok=True)
            with open("results/query_log.txt", "a") as f:
                f.write(json.dumps(query_log) + "\n")

            print("Evaluation log updated: results/query_log.txt")

            entities = result_data.get("entities", [])
            relationships = result_data.get("relationships", [])
            if entities:
                try:
                    plot_relationships(entities, relationships)
                except Exception as e:
                    print("Error while plotting relationships:", e)
            else:
                print("No entities to plot.")

            print("\n=== Final Answer ===")
            print(result_data.get("final_answer") or result_data.get("answer") or "No answer found.")

            print("\n=== Parsed Results ===")
            print(json.dumps(query_log, indent=2))
        else:
            print("No file uploaded.")

# Display
run_button.on_click(on_run_clicked)
display(widgets.VBox([file_picker, query_box, run_button, output]))
